# CCTPY CUDA 程序开发笔记

## hello-world

先用 hello-world 回顾一下利用纯 C 语言和 pychuda 开发 CUDA 程序

<pre>
#include &lt;stdio.h&gt;

// 需要从 CPU 端调用，让 GPU 执行的函数称为内核函数，需要用 __global__ 修饰
__global__ void kernel_function(){
    printf("hello, world! -- from gpu\n");
}

int main() {
    printf("hello, world! -- from cpu\n");

    // 调用内核函数，使用 <<<>>> 分配块和线程
    // 一个块内有多个线程，块内支持内存共享
    // 块和线程都是三维结构
    // 下面定义了 1*1*1 个块，1*1*1 个线程
    kernel_function<<<dim3(1,1,1),dim3(1,1,1)>>>();
}
</pre>

In [11]:
# -*- coding: utf-8 -*-

import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule

# 内核函数定义
mod = SourceModule("""
    // 需要从 CPU 端调用，让 GPU 执行的函数称为内核函数，需要用 __global__ 修饰
    __global__ void kernel_function(){
        printf("hello, world! -- from gpu\\n");
    }
""")

# 获取内核函数
kernel_function = mod.get_function("kernel_function")

# 调用内核函数
# grid 就是块数目定义，block 是块内内存
kernel_function(grid=(1,1,1), block=(1,1,1))
print("hello, pycuda! -- from python")

"""
注意：jupyter notebook 中没有打印 GPU 运行信息，可能是监听的 IO 流不对
在独立的 py 文件中，能收到 GPU 的打印
"""

hello, pycuda! -- from python


'\n注意：jupyter notebook 中没有打印 GPU 运行信息，可能是监听的 IO 流不对\n在独立的 py 文件中，能收到 GPU 的打印\n'

## 块和线程

利用 blockIdx.x 可以访问块 x 方向编号

利用 threadIdx.x 可以访问线程 x 方向编号

GPU 加速粒子跟踪，我的想法是：一个块负责一个束线的一个粒子

<pre>
#include &lt;stdio.h&gt;

// 需要从 CPU 端调用，让 GPU 执行的函数称为内核函数，需要用 __global__ 修饰
__global__ void kernel_function(){
    int bid = blockIdx.x;
    int tid = threadIdx.x;
    printf("bid=%d, tid=%d\n",bid,tid);
}

int main() {
    printf("hello, world! -- from cpu\n");

    // 调用内核函数，使用 <<<>>> 分配块和线程
    // 一个块内有多个线程，块内支持内存共享
    // 块和线程都是三维结构
    // 下面定义了 1*1*1 个块，1*1*1 个线程

    kernel_function<<<dim3(1,1,1),dim3(1,1,1)>>>();
    cudaDeviceSynchronize();
    printf("---------\n");
    kernel_function<<<dim3(3,1,1),dim3(1,1,1)>>>();
    cudaDeviceSynchronize();
    printf("---------\n");
    kernel_function<<<dim3(1,1,1),dim3(3,1,1)>>>();
    cudaDeviceSynchronize();
    printf("---------\n");
    kernel_function<<<dim3(2,1,1),dim3(2,1,1)>>>();
    cudaDeviceSynchronize();
    printf("---------\n");
}
</pre>

**程序输出**

<pre>
hello, world! -- from cpu
bid=0, tid=0
---------
bid=0, tid=0
bid=2, tid=0
bid=1, tid=0
---------
bid=0, tid=0
bid=0, tid=1
bid=0, tid=2
---------
bid=0, tid=0
bid=0, tid=1
bid=1, tid=0
bid=1, tid=1
---------
</pre>

# CUDA 简单运算

## 单个数加法


In [33]:
# -*- coding: utf-8 -*-

import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
import numpy as np

# 内核函数定义
mod = SourceModule("""
    __global__ void add(float *a, float *b,float *ret){
        *ret = *a + *b;
    }
""")


add = mod.get_function("add")

a = np.array([10.0], dtype=np.float32)
b = np.array([2], dtype=np.float32)
c = np.empty(1, dtype=np.float32)

add(drv.In(a), drv.In(b), drv.Out(c), grid=(1,1,1), block=(1,1,1))

print(c)

[12.]


## 向量加法

In [32]:
# -*- coding: utf-8 -*-

import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
import numpy as np

# 内核函数定义
mod = SourceModule("""
    __global__ void add(float *a, float *b,int *length,float *ret){
        int tid = threadIdx.x;
        if(tid < *length)
            ret[tid] = a[tid] + b[tid];
    }
""")


add = mod.get_function("add")

length = 5
a = np.random.rand(length).astype(np.float32)
b = np.random.rand(length).astype(np.float32)
c = np.empty_like(a)

add(drv.In(a), drv.In(b),drv.In(np.array([length],dtype=np.int32)), drv.Out(c), grid=(1,1,1), block=(length,1,1))

print(a,b,c,sep='\n')
print(f"差异：{c-a-b}")

[0.18910336 0.10802753 0.8828782  0.19864516 0.10315145]
[0.48107558 0.2623948  0.770267   0.56897557 0.9377996 ]
[0.67017895 0.3704223  1.6531452  0.76762074 1.040951  ]
差异：[0. 0. 0. 0. 0.]


# 规约运算（加法规约 == 求和）

第一步：把 block 内数据求和（两两分组求和）

### 问题 1 如何减少线程束内的分束？使用图中第二种规约方法

<img src="./img/C01规约运算减少线程束内的份束.jpg"></img>

### 问题 2 数据存放在哪？全局内存？共享内存？是否需要同步？

In [57]:
# block 内规约求和

# -*- coding: utf-8 -*-

import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
import numpy as np
import time

# 内核函数定义
mod = SourceModule("""
    __global__ void block_summer(int* array,int* block_sum){
        int local_id = threadIdx.x; // block 内 tid
        int global_id = blockDim.x * blockIdx.x + local_id; // 全局 tid
        __shared__ int s[1024]; // block 内共享内存

        s[local_id] = array[global_id]; // 复制到共享内存
        __syncthreads(); // 块内同步

        for(int step = blockDim.x>>1; step >= 1; step>>=1 ){ // 跨步初始化为 block 大小的一半，并且每次减半
            if(local_id < step) s[local_id] += s[local_id+step]; // 跨步规约
        }

        if(local_id == 0) block_sum[blockIdx.x] = s[0]; // 每个块规约结束，则局部和放在了 s[0] 中
        // 这一步不用同步，应为最后一次加法执行人，就是 local_id == 0 
    }
""")


block_summer = mod.get_function("block_summer")

block_size = 1024
grid_size = 24000

array = np.random.randint(100,size=block_size*grid_size)
block_sum = np.empty((grid_size,),dtype=np.int32)

s1 = time.time()
block_summer(drv.In(array), drv.Out(block_sum), grid=(grid_size,1,1), block=(block_size,1,1))

s2 = time.time()
block_sum_cpu = np.sum(array.reshape(grid_size,block_size),axis=1)
e = time.time()

print(f"原数组{array}")
print(f"CUDA={block_sum}, CPU={block_sum_cpu}")
if np.sum(np.abs(block_sum-block_sum_cpu))==0:
    print("计算正确")
print(f"CPU用时{e-s2}秒, GPU用时{s2-s1}秒")


原数组[ 2 10 62 ... 32 97 47]
CUDA=[51746 50427 50581 ... 49424 50193 51345], CPU=[51746 50427 50581 ... 49424 50193 51345]
计算正确
CPU用时0.011775016784667969秒, GPU用时0.0263516902923584秒


In [64]:
# 全局求和，和上代码不同的仅仅是把  if(local_id == 0) block_sum[blockIdx.x] = s[0]; 
# 改成 if(local_id == 0) atomicAdd(global_sum, s[0]);

# block 内规约求和

# -*- coding: utf-8 -*-

import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
import numpy as np
import time

# 内核函数定义
mod = SourceModule("""
    __global__ void sum(int* array,int* global_sum){
        int local_id = threadIdx.x; // block 内 tid
        int global_id = blockDim.x * blockIdx.x + local_id; // 全局 tid
        __shared__ int s[1024]; // block 内共享内存

        s[local_id] = array[global_id]; // 复制到共享内存
        __syncthreads(); // 块内同步

        for(int step = blockDim.x>>1; step >= 1; step>>=1 ){ // 跨步初始化为 block 大小的一半，并且每次减半
            if(local_id < step) s[local_id] += s[local_id+step]; // 跨步规约
        }

        if(local_id == 0) atomicAdd(global_sum, s[0]);
    }
""")


sum = mod.get_function("sum")

block_size = 1024
grid_size = 24000

array = np.random.randint(100,size=block_size*grid_size)
global_sum = np.zeros((1,),dtype=np.int32)

s1 = time.time()
sum(drv.In(array), drv.Out(global_sum), grid=(grid_size,1,1), block=(block_size,1,1))

s2 = time.time()
global_sum_cpu = np.sum(array)
e = time.time()

print(f"原数组{array}")
print(f"CUDA={global_sum}, CPU={global_sum_cpu}")
if np.sum(np.abs(block_sum-block_sum_cpu))==0:
    print("计算正确")
print(f"CPU用时{e-s2}秒, GPU用时{s2-s1}秒")


原数组[73 19 10 ... 66  6  1]
CUDA=[1216424266], CPU=1216424266
计算正确
CPU用时0.00976109504699707秒, GPU用时0.023424148559570312秒


# 重要知识点

- __syncthreads() block 内线程同步，一般用于对 __share__ 变量的写读之间

- 全一维下线程全局 id = blockDim.x + blockIdx.x + threadIdx.x

- __shared__ 表示 block 内的线程共享变量

- 原子操作 atomic add / sub / ...